# BBC Weather API

In [6]:
from urllib.parse import urlencode
import requests

In [7]:
# While we search Chennai, this API call is triggered
# Using this API call, we cant get the id for chennai
response = requests.get(url="https://locator-service.api.bbci.co.uk/locations?api_key=AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv&stack=aws&locale=en&filter=international&place-types=settlement%2Cairport%2Cdistrict&order=importance&s=Chennai&a=true&format=json")
data = response.json()
data

{'response': {'results': {'results': [{'id': '1264527',
     'name': 'Chennai',
     'container': 'India',
     'containerId': 1269750,
     'language': 'en',
     'timezone': 'Asia/Kolkata',
     'country': 'IN',
     'latitude': 13.08784,
     'longitude': 80.27847,
     'placeType': 'settlement'},
    {'id': '6301127',
     'name': 'Chennai International Airport',
     'container': 'India',
     'containerId': 1269750,
     'language': 'en',
     'timezone': 'Asia/Kolkata',
     'country': 'IN',
     'latitude': 12.98833,
     'longitude': 80.16578,
     'placeType': 'airport'}],
   'totalResults': 2}}}

In [10]:
# id for chennai
data['response']['results']['results'][0]['id']

'1264527'

In [12]:
# convert the steps into a function, and make location as the parameter 
def get_id(location):
    '''
    returns the id of the location
    '''
    base_url_id = "https://locator-service.api.bbci.co.uk/locations?" + urlencode({
        "api_key":"AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv",
        "stack":"aws",
        "locale":"en",
        "filter":"international",
        "place-types":"settlement, airport, district",
        "order":"importance",
        "s":location,
        "a":"true",
        "format":"json"
    })
    response = requests.get(url=base_url_id)
    data = response.json()
    
    return data['response']['results']['results'][0]['id']

In [13]:
get_id("Kolkata") # gets the id for Kolkata

'1275004'

In [14]:
get_id("London") # gets the id for London

'2643743'

In [18]:
# now that we have the id, let us get the weather data for a location
# say, the location is Kolkata, India 
import json

id = get_id('Kolkata')
base_url = f"https://weather-broker-cdn.api.bbci.co.uk/en/forecast/aggregated/{id}"

response = requests.get(url=base_url)

if response.status_code == 200:
    data = response.json()
    
    # Directly write the prettified JSON data to a file
    with open('./JSON/weather_data_kolkata.json', 'w') as f:
        json.dump(data, f, indent=4)  # indent=4 for pretty formatting
    
    print("Data has been saved to 'weather_data_kolkata.json'")
else:
    print(f"Error: {response.status_code}")

Data has been saved to 'weather_data_kolkata.json'


In [19]:
data.keys() # the keys of the json

dict_keys(['forecasts', 'isNight', 'issueDate', 'lastUpdated', 'location', 'message', 'night'])

In [21]:
# convert this data to a pandas dataframe

import pandas as pd

# Load the JSON data from the file
with open('./JSON/weather_data_kolkata.json', 'r') as f:
    data = json.load(f)

# Extracting the 'reports' from the detailed section
reports = []

# Looping through forecasts and collecting reports data
for forecast in data['forecasts']:
    for report in forecast['detailed']['reports']:
        reports.append({
            'localDate': report['localDate'],
            'timeslot': report['timeslot'],
            'temperatureC': report['temperatureC'],
            'feelsLikeTemperatureC': report['feelsLikeTemperatureC'],
            'humidity': report['humidity'],
            'precipitationProbability': report['precipitationProbabilityInPercent'],
            'windSpeedKph': report['windSpeedKph'],
            'weatherDescription': report['enhancedWeatherDescription']
        })

# Convert the list of reports to a DataFrame
df = pd.DataFrame(reports)

# Show the DataFrame
print(df.head())

# Save to CSV if needed
df.to_csv('./CSV/weather_data_kolkata.csv', index=False)

    localDate timeslot  temperatureC  feelsLikeTemperatureC  humidity  \
0  2024-10-10    20:30            28                     39        86   
1  2024-10-10    21:30            28                     39        85   
2  2024-10-10    22:30            28                     39        85   
3  2024-10-10    23:30            27                     39        89   
4  2024-10-11    00:30            27                     39        87   

   precipitationProbability  windSpeedKph     weatherDescription  
0                         3         18380  Hurricane force winds  
1                         2         18380  Hurricane force winds  
2                         2         18380  Hurricane force winds  
3                         2         18380  Hurricane force winds  
4                         0         18380  Hurricane force winds  


In [22]:
# write a function to generalize this for any location

def weather_data_in_csv(location):
    id = get_id(location=location)
    base_url = f"https://weather-broker-cdn.api.bbci.co.uk/en/forecast/aggregated/{id}"

    response = requests.get(url=base_url)

    if response.status_code == 200:
        data = response.json()
        
        # Directly write the prettified JSON data to a file
        file_path = f"./JSON/weather_data_{location}.json"
        with open(file_path, 'w') as f:
            json.dump(data, f, indent=4)  # indent=4 for pretty formatting
        
        print(f"Data has been saved to {file_path}")
    else:
        print(f"Error: {response.status_code}")
        
    # convert this data to a pandas dataframe

    # Load the JSON data from the file
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Extracting the 'reports' from the detailed section
    reports = []

    # Looping through forecasts and collecting reports data
    for forecast in data['forecasts']:
        for report in forecast['detailed']['reports']:
            reports.append({
                'localDate': report['localDate'],
                'timeslot': report['timeslot'],
                'temperatureC': report['temperatureC'],
                'feelsLikeTemperatureC': report['feelsLikeTemperatureC'],
                'humidity': report['humidity'],
                'precipitationProbability': report['precipitationProbabilityInPercent'],
                'windSpeedKph': report['windSpeedKph'],
                'weatherDescription': report['enhancedWeatherDescription']
            })

    # Convert the list of reports to a DataFrame
    df = pd.DataFrame(reports)

    # Show the DataFrame
    print(df.head())

    # Save to CSV if needed
    csv_path = f"./CSV/weather_data_{location}.csv"
    df.to_csv(csv_path, index=False)

In [23]:
# call the function for London
weather_data_in_csv(location='London')

Data has been saved to ./JSON/weather_data_London.json
    localDate timeslot  temperatureC  feelsLikeTemperatureC  humidity  \
0  2024-10-10    16:00            12                      7        57   
1  2024-10-10    17:00            11                      6        57   
2  2024-10-10    18:00            10                      5        60   
3  2024-10-10    19:00            10                      4        63   
4  2024-10-10    20:00             9                      2        68   

   precipitationProbability  windSpeedKph     weatherDescription  
0                         0           185  Hurricane force winds  
1                         0           185  Hurricane force winds  
2                         0           185  Hurricane force winds  
3                         0           185  Hurricane force winds  
4                         0           185  Hurricane force winds  


In [24]:
# let's now call it for Dubai
weather_data_in_csv(location='Dubai')

Data has been saved to ./JSON/weather_data_Dubai.json
    localDate timeslot  temperatureC  feelsLikeTemperatureC  humidity  \
0  2024-10-10    19:00            33                     42        56   
1  2024-10-10    20:00            32                     42        60   
2  2024-10-10    21:00            31                     42        65   
3  2024-10-10    22:00            31                     42        68   
4  2024-10-10    23:00            30                     41        72   

   precipitationProbability  windSpeedKph     weatherDescription  
0                         0         21624  Hurricane force winds  
1                         0         21624  Hurricane force winds  
2                         0         21624  Hurricane force winds  
3                         0         21624  Hurricane force winds  
4                         0         21624  Hurricane force winds  


Go to the `CSV` folder and the `JSON` folder to view the scraped data from the weather API!